# Notebook de smoke tests et automatisations
Ce notebook automatise:
- la résolution non-interactive des ports (backend/ui/jupyter/vscode),
- l’uniformisation du wording « Application » (codemod contrôlé),
- la journalisation structurée,
- un smoke test API pour les templates avec tags.

In [ ]:
# 1) Importer les dépendances
import os, sys, json, re, socket, logging, hashlib
from pathlib import Path
from dataclasses import dataclass
import argparse
try:
    import requests
except ImportError:
    raise SystemExit("Veuillez installer requests (pip install requests)")
# Seed logging de base
logging.basicConfig(level=logging.INFO, format='[%(levelname)s] %(message)s')
ROOT = Path(__file__).resolve().parents[2] if '__file__' in globals() else Path.cwd().parents[0]

In [ ]:
# 2) Modèle de configuration (Pydantic‑like minimal via dataclass) et valeurs par défaut
@dataclass
class Config:
    api_port: int = int(os.getenv('PORT_API', '8000'))
    ui_port: int = int(os.getenv('PORT_UI', '3000'))
    jupyter_port: int = int(os.getenv('JUPYTER_PORT', '8888'))
    vscode_port: int = int(os.getenv('VSCODE_PORT', '8080'))
    strict_ports: bool = bool(int(os.getenv('STRICT_PORTS', '0')))
CFG = Config()
CFG

In [ ]:
# 3) Backend base URL et session avec cookies
BASE_URL = os.getenv('BASE_URL', 'http://localhost:8000')
AUTH_LOGIN = f"{BASE_URL}/api/v1/auth/login"
AUTH_ME = f"{BASE_URL}/api/v1/auth/me"
STATUS = f"{BASE_URL}/api/v1/status"
HEALTH = f"{BASE_URL}/api/v1/health"
TPL_BASE = f"{BASE_URL}/api/v1/k8s/templates"
s = requests.Session()
s.headers.update({'Content-Type': 'application/json'})
BASE_URL

In [ ]:
# 4) Helpers HTTP
def jprint(title, data):
    print(f"\n=== {title} ===")
    print(json.dumps(data, indent=2, ensure_ascii=False))

def assert_status(resp, code=200):
    assert resp.status_code == code, f"HTTP {resp.status_code}: {resp.text}"
    try:
        return resp.json()
    except Exception:
        return {}

def login(username='admin', password='admin123'):
    payload = {"username": username, "password": password}
    r = s.post(AUTH_LOGIN, data=json.dumps(payload))
    data = assert_status(r, 200)
    # Session cookie devrait être posé par le backend
    # On vérifie /me pour confirmer
    r_me = s.get(AUTH_ME)
    assert r_me.status_code == 200, f"Login ko: {r_me.text}"
    return data

In [ ]:
# 5) Smoke tests: status/health + login
print("BASE_URL:", BASE_URL)
r = s.get(STATUS)
status_json = assert_status(r, 200)
jprint("/status", status_json)
r = s.get(HEALTH)
health_json = assert_status(r, 200)
jprint("/health", health_json)
login_data = login()
jprint("login", login_data)

In [ ]:
# 6) CRUD Templates avec tags
import random
suffix = str(random.randint(1000, 9999))
tpl_payload = {
    "key": f"demo-{suffix}",
    "name": f"Démo {suffix}",
    "description": "Template de test créé par smoke test",
    "icon": "🧪",
    "deployment_type": "custom",
    "default_image": "nginx:alpine",
    "default_port": 8080,
    "default_service_type": "NodePort",
    "active": True,
    "tags": ["demo", "test", "smoke"]
}
r = s.post(TPL_BASE, data=json.dumps(tpl_payload))
created = assert_status(r, 200)
tpl_id = created.get("id")
assert tpl_id, f"Pas d'id retourné: {created}"
jprint("template créé", created)

# GET all (admin)
r = s.get(f"{TPL_BASE}/all")
all_tpl = assert_status(r, 200)
assert any(t["id"] == tpl_id for t in all_tpl), "Template nouvellement créé introuvable dans /templates/all"
print(f"Total templates: {len(all_tpl)}")

# PUT: modifier tags
new_tags = ["demo", "updated", suffix]
r = s.put(f"{TPL_BASE}/{tpl_id}", data=json.dumps({"tags": new_tags}))
updated = assert_status(r, 200)
assert updated["tags"] == new_tags, f"Tags non mis à jour: {updated['tags']}"
jprint("template mis à jour", updated)

# DELETE
r = s.delete(f"{TPL_BASE}/{tpl_id}")
assert_status(r, 200)
print("Suppression OK")

### Notes
- Ce notebook suppose un backend en local: BASE_URL=http://localhost:8000.
- Identifiants par défaut: admin/admin123 (créés au démarrage si nécessaires).
- Les templates sont étendus avec `tags` (liste), utilisés ici pour valider le CRUD complet.
- Les ports par défaut pour vscode/jupyter sont auto-résolus côté backend (pas testés ici).